In [ ]:
pip install -U peft transformers accelerate


In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from peft import PeftModel

NLLB_BASE    = "facebook/nllb-200-distilled-1.3B"
NLLB_ADAPTER = "HassnaaElshafei/nllb_adapter"
SRC_LANG     = "ary_Arab"
TGT_LANG     = "arz_Arab"

device = "cuda" if torch.cuda.is_available() else "cpu"

tok = AutoTokenizer.from_pretrained(NLLB_BASE)
base = AutoModelForSeq2SeqLM.from_pretrained(NLLB_BASE).to(device)
model = PeftModel.from_pretrained(base, NLLB_ADAPTER).to(device).eval()

def nllb_darija_to_egy(text, max_length=128, num_beams=5):
    tok.src_lang = SRC_LANG
    inputs = tok(text, return_tensors="pt").to(device)
    forced_bos = tok.convert_tokens_to_ids(TGT_LANG)
    out = model.generate(**inputs, forced_bos_token_id=forced_bos, max_length=max_length, num_beams=num_beams)
    return tok.decode(out[0], skip_special_tokens=True).strip()


Loading weights:   0%|          | 0/1016 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie model.shared.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
The tied weights mapping and config for this model specifies to tie model.shared.weight to model.decoder.embed_tokens.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
The tied weights mapping and config for this model specifies to tie model.shared.weight to model.encoder.embed_tokens.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


model.safetensors:   0%|          | 0.00/5.48G [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/189M [00:00<?, ?B/s]

In [2]:
moroccan_sentences = [
    "بغيت نمشي للدار دابا.",
    "هاد التليفون ديالي ماشي ديالك.",
    "توحشتك بزاف يا خويا.",
    "غادي نسافر لفرانسا السيمانة الجاية.",
    "ما بغيتش ناكل، شبعت.",
    "واش كاين شي جديد؟",
    "فين غادي تمشي ف العشية؟",
    "شحال داير هاد السروال؟",
    "عطيني شي كاس ديال الماء عفاك.",
    "الدراري كيلعبو الكورة ف الزنقة."
]

print("--- 🇲🇦 Darija -> 🇪🇬 Egyptian Using Finetuned NLLB ---")
for sent in moroccan_sentences:
    print(f"\nInput:  {sent}")
    # Use your function
    output = nllb_darija_to_egy(sent)
    print(f"Output: {output}")

--- 🇲🇦 Darija -> 🇪🇬 Egyptian Using Finetuned NLLB ---

Input:  بغيت نمشي للدار دابا.
Output: عايز اروح البيت دلوقتي .

Input:  هاد التليفون ديالي ماشي ديالك.
Output: ده تليفوني مش بتاعك.

Input:  توحشتك بزاف يا خويا.
Output: لوسيتك جامد او وحش .

Input:  غادي نسافر لفرانسا السيمانة الجاية.
Output: حأسافر لفرنسا الاسبوع الجاي .

Input:  ما بغيتش ناكل، شبعت.
Output: أنا مش عايز آكل، أنا شتيت.

Input:  واش كاين شي جديد؟
Output: فيه اى حاجه جديدة؟

Input:  فين غادي تمشي ف العشية؟
Output: هتروح فين بليل؟

Input:  شحال داير هاد السروال؟
Output: بكام البنطلون ده؟

Input:  عطيني شي كاس ديال الماء عفاك.
Output: إديني كوبايه ميه لو سمحت.

Input:  الدراري كيلعبو الكورة ف الزنقة.
Output: الدراري بيلعبوا كرة قدم في الشارع .
